In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [26]:
# verify narrow version
import time

def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    for a in [1..n+1]:
        d=order(2,a)
        purepart=2^d
        oddpart=a/2^d
        #ord(2,tau(2^n))= (I believe) 3d
        # so ord(2,tau(purepart))=ord(2,tau(2^d)) = 3n
        # so tau(purepart) = 2^(3d)
        # so tau(a) = tau(purepart)*tau(oddpart)=
        # 8^d*tau(oddpart); then we replace tau(oddpart)
        # with sigma(oddpart,k=11) by Ramanujan's congruence
        term=8^d*sigma(oddpart,k=11)
        ans=ans+term*x^a
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

ls=[]
tests=[]
residues=[]
start=time.time()
for a in [1..1000]: #default precision must be > max a.
    c=p2RecipPowerConstantTerm(a)
    if c==0:
        ls=ls+[a]
        print((a,"c = 0"))
    if c!=0:
        o3=order(3,c)
        mc=mod(c/3^o3,3)
        residues=residues+[mc]
        tests=tests+[o3]
        #ds=digitsum(3,a)
        finish=time.time()
        print((a,o3,mc,finish-start))
print()
print("no-count:");print();print(len(ls))
print("bad a's:");print();print(ls)
print();print("tests:");print();print(tests)
print();print("residues");print();print(residues)
print();print(sortAndSmash(residues))

(1, 0, 1, 0.006438255310058594)
(2, 0, 1, 0.028060197830200195)
(3, 0, 1, 0.05341911315917969)
(4, 0, 1, 0.08431005477905273)
(5, 0, 1, 0.11953020095825195)
(6, 0, 1, 0.17796921730041504)
(7, 0, 1, 0.24185514450073242)
(8, 0, 1, 0.302579402923584)
(9, 0, 1, 0.3633902072906494)
(10, 0, 1, 0.42374229431152344)
(11, 0, 1, 0.4863111972808838)
(12, 0, 1, 0.548626184463501)
(13, 0, 1, 0.6111631393432617)
(14, 0, 1, 0.6736702919006348)
(15, 0, 1, 0.7357633113861084)
(16, 0, 1, 0.7977631092071533)
(17, 0, 1, 0.8601572513580322)
(18, 0, 1, 0.92417311668396)
(19, 0, 1, 0.9876370429992676)
(20, 0, 1, 1.0529911518096924)
(21, 0, 1, 1.1217763423919678)
(22, 0, 1, 1.1909871101379395)
(23, 0, 1, 1.2618613243103027)
(24, 0, 1, 1.3307571411132812)
(25, 0, 1, 1.4002432823181152)
(26, 0, 1, 1.4684662818908691)
(27, 0, 1, 1.5368731021881104)
(28, 0, 1, 1.6053130626678467)
(29, 0, 1, 1.6763720512390137)
(30, 0, 1, 1.7459721565246582)
(31, 0, 1, 1.8157382011413574)
(32, 0, 1, 1.928910255432129)
(33, 0, 1, 1

(265, 0, 1, 23.1845703125)
(266, 0, 1, 23.302364349365234)
(267, 0, 1, 23.42044425010681)
(268, 0, 1, 23.53665018081665)
(269, 0, 1, 23.657397031784058)
(270, 0, 1, 23.7759690284729)
(271, 0, 1, 23.895763397216797)
(272, 0, 1, 24.012718200683594)
(273, 0, 1, 24.131381034851074)
(274, 0, 1, 24.250307083129883)
(275, 0, 1, 24.373192310333252)
(276, 0, 1, 24.494611263275146)
(277, 0, 1, 24.619776010513306)
(278, 0, 1, 24.742196083068848)
(279, 0, 1, 24.868680238723755)
(280, 0, 1, 24.99111819267273)
(281, 0, 1, 25.115814208984375)
(282, 0, 1, 25.241111040115356)
(283, 0, 1, 25.367107152938843)
(284, 0, 1, 25.492239236831665)
(285, 0, 1, 25.620543003082275)
(286, 0, 1, 25.747974157333374)
(287, 0, 1, 25.877358198165894)
(288, 0, 1, 26.00685214996338)
(289, 0, 1, 26.1320641040802)
(290, 0, 1, 26.256193161010742)
(291, 0, 1, 26.384539365768433)
(292, 0, 1, 26.511754035949707)
(293, 0, 1, 26.641547203063965)
(294, 0, 1, 26.769164085388184)
(295, 0, 1, 26.899715185165405)
(296, 0, 1, 27.027280

(528, 0, 1, 73.11252021789551)
(529, 0, 1, 73.38815522193909)
(530, 0, 1, 73.66574621200562)
(531, 0, 1, 73.96723318099976)
(532, 0, 1, 74.24789214134216)
(533, 0, 1, 74.55050134658813)
(534, 0, 1, 74.85003614425659)
(535, 0, 1, 75.16712021827698)
(536, 0, 1, 75.44440412521362)
(537, 0, 1, 75.73895502090454)
(538, 0, 1, 76.0385012626648)
(539, 0, 1, 76.35308909416199)
(540, 0, 1, 76.64993834495544)
(541, 0, 1, 76.96493005752563)
(542, 0, 1, 77.2800121307373)
(543, 0, 1, 77.61361408233643)
(544, 0, 1, 77.87840223312378)
(545, 0, 1, 78.16342830657959)
(546, 0, 1, 78.44756412506104)
(547, 0, 1, 78.75461030006409)
(548, 0, 1, 79.03997421264648)
(549, 0, 1, 79.35473728179932)
(550, 0, 1, 79.67195534706116)
(551, 0, 1, 80.00924921035767)
(552, 0, 1, 80.31231212615967)
(553, 0, 1, 80.62592315673828)
(554, 0, 1, 80.934885263443)
(555, 0, 1, 81.26070308685303)
(556, 0, 1, 81.56764435768127)
(557, 0, 1, 81.89788222312927)
(558, 0, 1, 82.22295713424683)
(559, 0, 1, 82.57414627075195)
(560, 0, 1, 

(789, 0, 1, 180.52228927612305)
(790, 0, 1, 181.05097031593323)
(791, 0, 1, 181.6133849620819)
(792, 0, 1, 182.1037633419037)
(793, 0, 1, 182.63554310798645)
(794, 0, 1, 183.16300916671753)
(795, 0, 1, 183.71587920188904)
(796, 0, 1, 184.23621916770935)
(797, 0, 1, 184.79039120674133)
(798, 0, 1, 185.34392619132996)
(799, 0, 1, 185.93367314338684)
(800, 0, 1, 186.39617824554443)
(801, 0, 1, 186.89085221290588)
(802, 0, 1, 187.38623523712158)
(803, 0, 1, 187.9143590927124)
(804, 0, 1, 188.4115080833435)
(805, 0, 1, 188.94334530830383)
(806, 0, 1, 189.47505831718445)
(807, 0, 1, 190.04003620147705)
(808, 0, 1, 190.53735828399658)
(809, 0, 1, 191.0713562965393)
(810, 0, 1, 191.60472512245178)
(811, 0, 1, 192.17542719841003)
(812, 0, 1, 192.7123463153839)
(813, 0, 1, 193.28138518333435)
(814, 0, 1, 193.85337924957275)
(815, 0, 1, 194.46255326271057)
(816, 0, 1, 194.97811627388)
(817, 0, 1, 195.55135321617126)
(818, 0, 1, 196.13243913650513)
(819, 0, 1, 196.72099924087524)
(820, 0, 1, 197.2